# create main slicing list

In [1]:
import copy  # copy big/deep objects by value
import csv
import datetime  # datetime operations
import itertools  # operate with iterators
import json  # read/write from/into json format
import math
import os  # OS operations (read/write files/folders)
import sys
import time
import warnings  # hide warnings
from collections import Counter
from itertools import groupby
import matplotlib

# process parallelization
from multiprocessing import Manager, Pool, RawArray, cpu_count
from os.path import exists

import matplotlib.pyplot as plt  # mother of plots focr Python

# import mlxtend
import numpy as np  # array/matrix operations (e.g. linear algebra)
import pandas as pd  # operate with dataframes


import scipy.signal

In [2]:
datapath = "E:/Cyprus_project_overview/my_data_backup/eye tracking data_backup"
data_folders = sorted([f for f in os.listdir(datapath) if not f.startswith('TestS')], key=str.lower)
print(data_folders)

['Expl_1_ET_1_2023-09-05_11-56-16_ET', 'Expl_1_ET_2_2023-09-05_12-34-24_ET', 'Expl_1_ET_3_2023-09-05_13-10-01_ET', 'Expl_2_ET_1_2023-09-06_10-36-37_ET', 'Expl_2_ET_2_2023-09-06_11-08-36_ET', 'Expl_2_ET_3_2023-09-06_11-39-21_ET', 'Expl_3_ET_1_2023-09-06_13-24-43_ET', 'Expl_3_ET_2_2023-09-06_13-57-57_ET', 'Expl_3_ET_3_2023-09-06_14-28-39_ET', 'Expl_4_ET_1_2023-09-06_18-31-33_ET', 'Expl_4_ET_2_2023-09-06_18-57-24_ET', 'Expl_5_ET_1_2023-09-07_18-17-19_ET', 'Expl_5_ET_2_2023-09-07_18-48-26_ET']


In [3]:
# datapath = "E:/Cyprus_project_overview/data/ETdata/oldData/final_labeling_data_frame_nr_NOT_sorted"
slicingpath = "E:/Cyprus_project_overview/data/slicingLists"


In [4]:
slicingList = pd.read_csv(os.path.join(slicingpath, "overview_ET_slicing_w_ts.csv"))
slicingList.head()

,session,Paula_start,Paula_start_ns,fixation_adjusted_start,fixation_adjusted_start_ns,GPS_sync_start,GPS_sync_start_ns,Paula_end,Paula_end_ns,fixation_adjusted_end,fixation_adjusted_end_ns,GPS_sync_end,GPS_sync_end_ns
0,Expl_1_ET_1_2023-09-05_11-56-16_ET,13670,1693904632544755555,13670,1693904632544755555,13670,1693904632544755555,32920,1693905274388766666,32882,1693905273121522222,32923,1693905274488733333
1,Expl_1_ET_2_2023-09-05_12-34-24_ET,9880,1693906794272600000,9895,1693906794774588888,9893,1693906794704611111,28805,1693907425280677777,28751,1693907423480544444,28810,1693907425444700000
2,Expl_1_ET_3_2023-09-05_13-10-01_ET,8060,1693908870728877777,8078,1693908871328911111,8075,1693908871229344444,29530,1693909586593522222,29517,1693909586160911111,29530,1693909586593522222
3,Expl_2_ET_1_2023-09-06_10-36-37_ET,11600,1693986184403811111,11629,1693986185372233333,11613,1693986184835755555,30680,1693986820579733333,30668,1693986820179755555,30686,1693986820779733333
4,Expl_2_ET_2_2023-09-06_11-08-36_ET,8620,1693988004851122222,8627,1693988005087088888,8624,1693988004987155555,27960,1693988649695222222,27943,1693988649131100000,27979,1693988650331144444


In [5]:
mainSlicing = pd.read_csv(os.path.join(slicingpath, "mainSlicingList.csv"))
mainSlicing.head()

,file,session,ET,Paula_start,Paula_start_ns,Paula_end,Paula_end_ns,GPS_sync_start,GPS_sync_start_ns,GPS_sync_end,GPS_sync_end_ns,et_start,et_start_ns,et_end,et_end_ns,experimentStart_worldCFrame,experimentEnd_worldCFrame
0,Expl_1_ET_1_2023-09-05_11-56-16_ET,1,1,13670,1693904632544755555,32920,1693905274388766666,13670,1693904632544755555,32923,1693905274488733333,90779.0,1.693905e+18,218604.0,1.693905e+18,14509.0,32717.0
1,Expl_1_ET_2_2023-09-05_12-34-24_ET,1,2,9880,1693906794272600000,28805,1693907425280677777,9893,1693906794704611111,28810,1693907425444700000,65685.0,1.693907e+18,191257.0,1.693907e+18,10700.0,28625.0
2,Expl_1_ET_3_2023-09-05_13-10-01_ET,1,3,8060,1693908870728877777,29530,1693909586593522222,8075,1693908871229344444,29530,1693909586593522222,53609.0,1.693909e+18,196254.0,1.693910e+18,8940.0,29340.0
3,Expl_2_ET_1_2023-09-06_10-36-37_ET,2,1,11600,1693986184403811111,30680,1693986820579733333,11613,1693986184835755555,30686,1693986820779733333,76972.0,1.693986e+18,203626.0,1.693987e+18,12348.0,NaN
4,Expl_2_ET_2_2023-09-06_11-08-36_ET,2,2,8620,1693988004851122222,27960,1693988649695222222,8624,1693988004987155555,27979,1693988650331144444,57234.0,1.693988e+18,185769.0,1.693989e+18,NaN,NaN


In [6]:
# mainSlicing['experimentStart_worldCFrame'] = pd.Series([pd.NA] * len(mainSlicing), dtype="Int64")
# mainSlicing['experimentEnd_worldCFrame'] = pd.Series([pd.NA] * len(mainSlicing), dtype="Int64")
# mainSlicing.head()

In [33]:
indexM = 12
mainSlicing.at[indexM, 'experimentStart_worldCFrame'] = 8804
mainSlicing.head(indexM+2)

,file,session,ET,Paula_start,Paula_start_ns,Paula_end,Paula_end_ns,GPS_sync_start,GPS_sync_start_ns,GPS_sync_end,GPS_sync_end_ns,et_start,et_start_ns,et_end,et_end_ns,experimentStart_worldCFrame,experimentEnd_worldCFrame
0,Expl_1_ET_1_2023-09-05_11-56-16_ET,1,1,13670,1693904632544755555,32920,1693905274388766666,13670,1693904632544755555,32923,1693905274488733333,90779.0,1.693905e+18,218604.0,1.693905e+18,14509.0,32717.0
1,Expl_1_ET_2_2023-09-05_12-34-24_ET,1,2,9880,1693906794272600000,28805,1693907425280677777,9893,1693906794704611111,28810,1693907425444700000,65685.0,1.693907e+18,191257.0,1.693907e+18,10700.0,28625.0
2,Expl_1_ET_3_2023-09-05_13-10-01_ET,1,3,8060,1693908870728877777,29530,1693909586593522222,8075,1693908871229344444,29530,1693909586593522222,53609.0,1.693909e+18,196254.0,1.693910e+18,8940.0,29340.0
3,Expl_2_ET_1_2023-09-06_10-36-37_ET,2,1,11600,1693986184403811111,30680,1693986820579733333,11613,1693986184835755555,30686,1693986820779733333,76972.0,1.693986e+18,203626.0,1.693987e+18,12348.0,30622.0
4,Expl_2_ET_2_2023-09-06_11-08-36_ET,2,2,8620,1693988004851122222,27960,1693988649695222222,8624,1693988004987155555,27979,1693988650331144444,57234.0,1.693988e+18,185769.0,1.693989e+18,9402.0,27848.0
5,Expl_2_ET_3_2023-09-06_11-39-21_ET,2,3,12160,1693989967720244444,31450,1693990610896299999,12165,1693989967884288888,31457,1693990611128255555,80751.0,1.693990e+18,208819.0,1.693991e+18,12964.0,31450.0
6,Expl_3_ET_1_2023-09-06_13-24-43_ET,3,1,8890,1693996180867344444,28170,1693996823707288888,8900,1693996181199322222,28177,1693996823943277777,58985.0,1.693996e+18,187253.0,1.693997e+18,10036.0,28170.0
7,Expl_3_ET_2_2023-09-06_13-57-57_ET,3,2,8710,1693998168175677777,27720,1693998802015900000,8726,1693998168707433333,27728,1693998802283477777,57920.0,1.693998e+18,184248.0,1.693999e+18,9413.0,27671.0
8,Expl_3_ET_3_2023-09-06_14-28-39_ET,3,3,7640,1693999974960144444,26950,1694000618804288888,7653,1693999975392355555,26955,1694000618972355555,50805.0,1.694000e+18,178817.0,1.694001e+18,8411.0,26850.0
9,Expl_4_ET_1_2023-09-06_18-31-33_ET,4,1,8600,1694014580757844444,27820,1694015221735855555,8607,1694014580993900000,27820,1694015221735855555,56933.0,1.694015e+18,184654.0,1.694015e+18,9192.0,27761.0


In [51]:
mainSlicing.at[1, 'experimentEnd_worldCFrame'] = 28710
mainSlicing.head(indexM+2)

,file,session,ET,Paula_start,Paula_start_ns,Paula_end,Paula_end_ns,GPS_sync_start,GPS_sync_start_ns,GPS_sync_end,GPS_sync_end_ns,et_start,et_start_ns,et_end,et_end_ns,experimentStart_worldCFrame,experimentEnd_worldCFrame,experimentStart_worldCFrame_ns,experimentEnd_worldCFrame_ns
0,Expl_1_ET_1_2023-09-05_11-56-16_ET,1,1,13670,1693904632544755555,32920,1693905274388766666,13670,1693904632544755555,32923,1693905274488733333,90779.0,1.693905e+18,218604.0,1.693905e+18,14509.0,32717.0,1693904660521111111,1693905267621355555
1,Expl_1_ET_2_2023-09-05_12-34-24_ET,1,2,9880,1693906794272600000,28805,1693907425280677777,9893,1693906794704611111,28810,1693907425444700000,65685.0,1.693907e+18,191257.0,1.693907e+18,10700.0,28710.0,1693906821612611111,1693907419276755555
2,Expl_1_ET_3_2023-09-05_13-10-01_ET,1,3,8060,1693908870728877777,29530,1693909586593522222,8075,1693908871229344444,29530,1693909586593522222,53609.0,1.693909e+18,196254.0,1.693910e+18,8940.0,29340.0,1693908900068955555,1693909580256966666
3,Expl_2_ET_1_2023-09-06_10-36-37_ET,2,1,11600,1693986184403811111,30680,1693986820579733333,11613,1693986184835755555,30686,1693986820779733333,76972.0,1.693986e+18,203626.0,1.693987e+18,12348.0,30622.0,1693986209344011111,1693986818643888888
4,Expl_2_ET_2_2023-09-06_11-08-36_ET,2,2,8620,1693988004851122222,27960,1693988649695222222,8624,1693988004987155555,27979,1693988650331144444,57234.0,1.693988e+18,185769.0,1.693989e+18,9402.0,27848.0,1693988030927133333,1693988645963177777
5,Expl_2_ET_3_2023-09-06_11-39-21_ET,2,3,12160,1693989967720244444,31450,1693990610896299999,12165,1693989967884288888,31457,1693990611128255555,80751.0,1.693990e+18,208819.0,1.693991e+18,12964.0,31450.0,1693989994524300000,1693990610896299999
6,Expl_3_ET_1_2023-09-06_13-24-43_ET,3,1,8890,1693996180867344444,28170,1693996823707288888,8900,1693996181199322222,28177,1693996823943277777,58985.0,1.693996e+18,187253.0,1.693997e+18,10036.0,28170.0,1693996219075611111,1693996823707288888
7,Expl_3_ET_2_2023-09-06_13-57-57_ET,3,2,8710,1693998168175677777,27720,1693998802015900000,8726,1693998168707433333,27728,1693998802283477777,57920.0,1.693998e+18,184248.0,1.693999e+18,9413.0,27671.0,1693998191615244444,1693998800383222222
8,Expl_3_ET_3_2023-09-06_14-28-39_ET,3,3,7640,1693999974960144444,26950,1694000618804288888,7653,1693999975392355555,26955,1694000618972355555,50805.0,1.694000e+18,178817.0,1.694001e+18,8411.0,26850.0,1694000000668477777,1694000615468288888
9,Expl_4_ET_1_2023-09-06_18-31-33_ET,4,1,8600,1694014580757844444,27820,1694015221735855555,8607,1694014580993900000,27820,1694015221735855555,56933.0,1.694015e+18,184654.0,1.694015e+18,9192.0,27761.0,1694014600631866666,1694015219767888888


In [36]:
mainSlicing.to_csv(os.path.join(slicingpath,"mainSlicingList.csv"), index=False)

## now add world timestamps to the selected slicing indeces

In [43]:
mainSlicing['experimentStart_worldCFrame_ns'] = pd.Series([pd.NA] * len(mainSlicing), dtype="Int64")
mainSlicing['experimentEnd_worldCFrame_ns'] = pd.Series([pd.NA] * len(mainSlicing), dtype="Int64")
mainSlicing.head()

,file,session,ET,Paula_start,Paula_start_ns,Paula_end,Paula_end_ns,GPS_sync_start,GPS_sync_start_ns,GPS_sync_end,GPS_sync_end_ns,et_start,et_start_ns,et_end,et_end_ns,experimentStart_worldCFrame,experimentEnd_worldCFrame,experimentStart_worldCFrame_ns,experimentEnd_worldCFrame_ns
0,Expl_1_ET_1_2023-09-05_11-56-16_ET,1,1,13670,1693904632544755555,32920,1693905274388766666,13670,1693904632544755555,32923,1693905274488733333,90779.0,1.693905e+18,218604.0,1.693905e+18,14509.0,32717.0,<NA>,<NA>
1,Expl_1_ET_2_2023-09-05_12-34-24_ET,1,2,9880,1693906794272600000,28805,1693907425280677777,9893,1693906794704611111,28810,1693907425444700000,65685.0,1.693907e+18,191257.0,1.693907e+18,10700.0,28625.0,<NA>,<NA>
2,Expl_1_ET_3_2023-09-05_13-10-01_ET,1,3,8060,1693908870728877777,29530,1693909586593522222,8075,1693908871229344444,29530,1693909586593522222,53609.0,1.693909e+18,196254.0,1.693910e+18,8940.0,29340.0,<NA>,<NA>
3,Expl_2_ET_1_2023-09-06_10-36-37_ET,2,1,11600,1693986184403811111,30680,1693986820579733333,11613,1693986184835755555,30686,1693986820779733333,76972.0,1.693986e+18,203626.0,1.693987e+18,12348.0,30622.0,<NA>,<NA>
4,Expl_2_ET_2_2023-09-06_11-08-36_ET,2,2,8620,1693988004851122222,27960,1693988649695222222,8624,1693988004987155555,27979,1693988650331144444,57234.0,1.693988e+18,185769.0,1.693989e+18,9402.0,27848.0,<NA>,<NA>


In [52]:
for idx,file in enumerate(mainSlicing['file']):
    print(file)
    worldTS = pd.read_csv(os.path.join(datapath, file, "world_timestamps.csv"))
    # .at[indexM, 'experimentEnd_worldCFrame'] = 27650
    # print(worldTS.at[mainSlicing.at[idx,'experimentStart_worldCFrame'], 'timestamp [ns]'])
    mainSlicing.at[idx,'experimentStart_worldCFrame_ns'] = worldTS.at[mainSlicing.at[idx,'experimentStart_worldCFrame'], 'timestamp [ns]']
    mainSlicing.at[idx,'experimentEnd_worldCFrame_ns'] = worldTS.at[mainSlicing.at[idx,'experimentEnd_worldCFrame'], 'timestamp [ns]']
    
print('done')

    

Expl_1_ET_1_2023-09-05_11-56-16_ET
Expl_1_ET_2_2023-09-05_12-34-24_ET
Expl_1_ET_3_2023-09-05_13-10-01_ET
Expl_2_ET_1_2023-09-06_10-36-37_ET
Expl_2_ET_2_2023-09-06_11-08-36_ET
Expl_2_ET_3_2023-09-06_11-39-21_ET
Expl_3_ET_1_2023-09-06_13-24-43_ET
Expl_3_ET_2_2023-09-06_13-57-57_ET
Expl_3_ET_3_2023-09-06_14-28-39_ET
Expl_4_ET_1_2023-09-06_18-31-33_ET
Expl_4_ET_2_2023-09-06_18-57-24_ET
Expl_5_ET_1_2023-09-07_18-17-19_ET
Expl_5_ET_2_2023-09-07_18-48-26_ET
done


In [53]:
mainSlicing.head(13)

,file,session,ET,Paula_start,Paula_start_ns,Paula_end,Paula_end_ns,GPS_sync_start,GPS_sync_start_ns,GPS_sync_end,GPS_sync_end_ns,et_start,et_start_ns,et_end,et_end_ns,experimentStart_worldCFrame,experimentEnd_worldCFrame,experimentStart_worldCFrame_ns,experimentEnd_worldCFrame_ns
0,Expl_1_ET_1_2023-09-05_11-56-16_ET,1,1,13670,1693904632544755555,32920,1693905274388766666,13670,1693904632544755555,32923,1693905274488733333,90779.0,1.693905e+18,218604.0,1.693905e+18,14509.0,32717.0,1693904660521111111,1693905267621355555
1,Expl_1_ET_2_2023-09-05_12-34-24_ET,1,2,9880,1693906794272600000,28805,1693907425280677777,9893,1693906794704611111,28810,1693907425444700000,65685.0,1.693907e+18,191257.0,1.693907e+18,10700.0,28710.0,1693906821612611111,1693907422112644444
2,Expl_1_ET_3_2023-09-05_13-10-01_ET,1,3,8060,1693908870728877777,29530,1693909586593522222,8075,1693908871229344444,29530,1693909586593522222,53609.0,1.693909e+18,196254.0,1.693910e+18,8940.0,29340.0,1693908900068955555,1693909580256966666
3,Expl_2_ET_1_2023-09-06_10-36-37_ET,2,1,11600,1693986184403811111,30680,1693986820579733333,11613,1693986184835755555,30686,1693986820779733333,76972.0,1.693986e+18,203626.0,1.693987e+18,12348.0,30622.0,1693986209344011111,1693986818643888888
4,Expl_2_ET_2_2023-09-06_11-08-36_ET,2,2,8620,1693988004851122222,27960,1693988649695222222,8624,1693988004987155555,27979,1693988650331144444,57234.0,1.693988e+18,185769.0,1.693989e+18,9402.0,27848.0,1693988030927133333,1693988645963177777
5,Expl_2_ET_3_2023-09-06_11-39-21_ET,2,3,12160,1693989967720244444,31450,1693990610896299999,12165,1693989967884288888,31457,1693990611128255555,80751.0,1.693990e+18,208819.0,1.693991e+18,12964.0,31450.0,1693989994524300000,1693990610896299999
6,Expl_3_ET_1_2023-09-06_13-24-43_ET,3,1,8890,1693996180867344444,28170,1693996823707288888,8900,1693996181199322222,28177,1693996823943277777,58985.0,1.693996e+18,187253.0,1.693997e+18,10036.0,28170.0,1693996219075611111,1693996823707288888
7,Expl_3_ET_2_2023-09-06_13-57-57_ET,3,2,8710,1693998168175677777,27720,1693998802015900000,8726,1693998168707433333,27728,1693998802283477777,57920.0,1.693998e+18,184248.0,1.693999e+18,9413.0,27671.0,1693998191615244444,1693998800383222222
8,Expl_3_ET_3_2023-09-06_14-28-39_ET,3,3,7640,1693999974960144444,26950,1694000618804288888,7653,1693999975392355555,26955,1694000618972355555,50805.0,1.694000e+18,178817.0,1.694001e+18,8411.0,26850.0,1694000000668477777,1694000615468288888
9,Expl_4_ET_1_2023-09-06_18-31-33_ET,4,1,8600,1694014580757844444,27820,1694015221735855555,8607,1694014580993900000,27820,1694015221735855555,56933.0,1.694015e+18,184654.0,1.694015e+18,9192.0,27761.0,1694014600631866666,1694015219767888888


In [55]:
mainSlicing.to_csv(os.path.join(slicingpath,"mainSlicingList.csv"), index=False)

## sanity check new timestamps

In [58]:
# check experiment time length --> should be around 10 min each
testTimeStamps = mainSlicing['experimentEnd_worldCFrame_ns'] - mainSlicing['experimentStart_worldCFrame_ns']
print((testTimeStamps/1e9)/60)

0     10.118337
1     10.008334
2     11.336467
3     10.154998
4     10.250601
5     10.272867
6     10.077195
7     10.146133
8     10.246664
9     10.318934
10    10.243333
11    10.197265
12    10.472868
dtype: Float64


## now save old et timestamps, so I can compare them, if I want later

In [72]:
mainSlicing['Old_et_start'] = mainSlicing['et_start'] 
mainSlicing['Old_et_start_ns'] = mainSlicing['et_start_ns'] 
mainSlicing['Old_et_end'] = mainSlicing['et_end'] 
mainSlicing['Old_et_end_ns'] = mainSlicing['et_end_ns'] 

mainSlicing['et_start_ns'] = pd.Series([pd.NA] * len(mainSlicing), dtype="Int64")
mainSlicing['et_end_ns'] = pd.Series([pd.NA] * len(mainSlicing), dtype="Int64")

mainSlicing.head()

,file,session,ET,Paula_start,Paula_start_ns,Paula_end,Paula_end_ns,GPS_sync_start,GPS_sync_start_ns,GPS_sync_end,GPS_sync_end_ns,et_start,et_start_ns,et_end,et_end_ns,experimentStart_worldCFrame,experimentEnd_worldCFrame,experimentStart_worldCFrame_ns,experimentEnd_worldCFrame_ns,Old_et_start,Old_et_start_ns,Old_et_end,Old_et_end_ns
0,Expl_1_ET_1_2023-09-05_11-56-16_ET,1,1,13670,1693904632544755555,32920,1693905274388766666,13670,1693904632544755555,32923,1693905274488733333,96341.0,<NA>,218604.0,<NA>,14509.0,32717.0,1693904660521111111,1693905267621355555,96341.0,1.693905e+18,218604.0,1.693905e+18
1,Expl_1_ET_2_2023-09-05_12-34-24_ET,1,2,9880,1693906794272600000,28805,1693907425280677777,9893,1693906794704611111,28810,1693907425444700000,65685.0,<NA>,191257.0,<NA>,10700.0,28710.0,1693906821612611111,1693907422112644444,65685.0,1.693907e+18,191257.0,1.693907e+18
2,Expl_1_ET_3_2023-09-05_13-10-01_ET,1,3,8060,1693908870728877777,29530,1693909586593522222,8075,1693908871229344444,29530,1693909586593522222,53609.0,<NA>,196254.0,<NA>,8940.0,29340.0,1693908900068955555,1693909580256966666,53609.0,1.693909e+18,196254.0,1.693910e+18
3,Expl_2_ET_1_2023-09-06_10-36-37_ET,2,1,11600,1693986184403811111,30680,1693986820579733333,11613,1693986184835755555,30686,1693986820779733333,76972.0,<NA>,203626.0,<NA>,12348.0,30622.0,1693986209344011111,1693986818643888888,76972.0,1.693986e+18,203626.0,1.693987e+18
4,Expl_2_ET_2_2023-09-06_11-08-36_ET,2,2,8620,1693988004851122222,27960,1693988649695222222,8624,1693988004987155555,27979,1693988650331144444,57234.0,<NA>,185769.0,<NA>,9402.0,27848.0,1693988030927133333,1693988645963177777,57234.0,1.693988e+18,185769.0,1.693989e+18


## now select the proper et indeces and timestamps and double check each of them

In [110]:
# load the labled data to check the affected/cut fixations when necessary

labledDataPath = "E:/Cyprus_project_overview/store_cyprus_project/pvondrlik/data/results_cyprus"
# allLabledData = pd.read_csv(os.path.join(labledDataPath, "fixation_and_labels_total.csv"))
allLabledData = pd.read_csv(os.path.join(labledDataPath, "fix_only_label_total.csv"))
allLabledData.head()

,session,session_duration,fixation_id,start_timestamp_ns,end_timestamp_ns,duration_ms,fixation_x,fixation_y,fixation_sum_label,fixation_pXc_label,level_sum_annotation,level_pXc_annotation,is_outlier
0,Expl_1_ET_1_2023-09-05_11-56-16_ET,6.416136e+11,964,1693904632719606803,1693904632794434803,74,1188.792,861.039,street,street,1.0,1.0,False
1,Expl_1_ET_1_2023-09-05_11-56-16_ET,6.416136e+11,965,1693904632860034803,1693904632969623803,109,1228.189,799.368,street,street,3.0,3.0,False
2,Expl_1_ET_1_2023-09-05_11-56-16_ET,6.416136e+11,966,1693904633019796803,1693904633135001803,115,1184.857,710.476,street,street,1.0,1.0,False
3,Expl_1_ET_1_2023-09-05_11-56-16_ET,6.416136e+11,967,1693904633310056803,1693904633450633803,140,1156.521,806.834,street,street,1.0,1.0,False
4,Expl_1_ET_1_2023-09-05_11-56-16_ET,6.416136e+11,968,1693904633485040803,1693904633616352803,131,1015.828,729.272,street,street,1.0,1.0,False


## functions

In [60]:
def closest_boundary_index(df,
                           idx,
                           col="fixation id"):
    """
    Find the nearest row (in either direction) whose ``col`` value is
    NaN **or** different from the value at ``idx``.

    Parameters
    ----------
    df   : pd.DataFrame
        The DataFrame that contains the column ``col``.
    idx  : int
        The *position* (integer location) from which the search starts.
        (If you work with a label‑based index, use ``df.index.get_loc(idx)`` first.)
    col  : str, optional
        Column that holds the fixation id.

    Returns
    -------
    int or tuple or None
        The nearest index (or both indices) that satisfies the condition.
    """
    # ------------------------------------------------------------------
    # 1️⃣  Grab the reference value at ``idx`` – we already know it is not NaN
    # ------------------------------------------------------------------
    ref_val = df.iat[idx, df.columns.get_loc(col)]

    # ------------------------------------------------------------------
    # 2️⃣  Build a boolean mask of all rows that are *different* from ref_val
    #     OR are NaN.  ``pd.isna`` works for ``np.nan`` *and* ``pd.NA``.
    # ------------------------------------------------------------------
    mask = (df[col] != ref_val) | df[col].isna()

    # Convert the mask to a NumPy array of positions (integer locations)
    # ``np.where`` returns a tuple, the first element holds the indices.
    candidate_positions = np.where(mask.values)[0]

    # ------------------------------------------------------------------
    # 3️⃣  Split the candidates into “before” and “after” the start index
    # ------------------------------------------------------------------
    before = candidate_positions[candidate_positions < idx]
    after  = candidate_positions[candidate_positions > idx]

    # The nearest index on each side (or None if that side is empty)
    back_idx   = before.max() if before.size > 0 else None
    forward_idx = after.min() if after.size > 0 else None


    # Both exist → compare distances
    if (idx - back_idx) <= (forward_idx - idx):
        closestI = back_idx
    else:
        closestI = forward_idx

    return back_idx, forward_idx, closestI



# start slicing check overview loop here

In [70]:
# Show all columns, no matter how many there are
pd.set_option('display.max_columns', None)   # or use 0 or np.inf in newer pandas

In [279]:
index  = 10

print(mainSlicing.iloc[index]['file'])

fullGaze = pd.read_csv(os.path.join(datapath, mainSlicing.iloc[index]['file'], "gaze.csv"))

fullGaze.head()


Expl_4_ET_2_2023-09-06_18-57-24_ET


,section id,recording id,timestamp [ns],gaze x [px],gaze y [px],worn,fixation id,blink id,azimuth [deg],elevation [deg]
0,18a4e6ae-bb0f-4998-9c6a-9d8378364765,82923e39-34c1-42d3-af2e-5b1c2c24660b,1694015846741470994,723.496,452.786,1.0,1.0,NaN,-6.340453,9.877993
1,18a4e6ae-bb0f-4998-9c6a-9d8378364765,82923e39-34c1-42d3-af2e-5b1c2c24660b,1694015846744610994,723.248,452.814,1.0,1.0,NaN,-6.356472,9.876044
2,18a4e6ae-bb0f-4998-9c6a-9d8378364765,82923e39-34c1-42d3-af2e-5b1c2c24660b,1694015846750842994,720.871,452.885,1.0,1.0,NaN,-6.510359,9.870358
3,18a4e6ae-bb0f-4998-9c6a-9d8378364765,82923e39-34c1-42d3-af2e-5b1c2c24660b,1694015846753940994,718.031,452.974,1.0,1.0,NaN,-6.694199,9.863239
4,18a4e6ae-bb0f-4998-9c6a-9d8378364765,82923e39-34c1-42d3-af2e-5b1c2c24660b,1694015846760145994,718.819,453.633,1.0,1.0,NaN,-6.642678,9.821480


## do start slicing first

In [258]:
baseStart_TS = mainSlicing.at[index,'experimentStart_worldCFrame_ns']

mask1 = fullGaze["timestamp [ns]"] >= baseStart_TS
index_start = mask1.idxmax()          # returns the position of the **first True**

print(index_start)

if abs(baseStart_TS - fullGaze.iloc[index_start]['timestamp [ns]']) <= abs(baseStart_TS - fullGaze.iloc[index_start-1]['timestamp [ns]']):
    print("keep current index and timestamp")

else: 
    print("row earlier (index -1) is actually closer to the world ts")
    index_start = index_start-1
    print(index_start)



58407
keep current index and timestamp


In [259]:
if pd.notna(fullGaze.iloc[index_start]["fixation id"]):
    print("fixation ID at start index---------------")
    print(fullGaze.iloc[index_start]["fixation id"])
    back_idx, forward_idx, closestI = closest_boundary_index(fullGaze, idx=index_start, col="fixation id")
    print(back_idx, forward_idx, closestI)

    print("differences back idx minus index_start", back_idx - index_start)
    print("differences forward idx minus index_start", forward_idx - index_start)

    if abs(fullGaze.iloc[back_idx]["timestamp [ns]"] - fullGaze.iloc[index_start]["timestamp [ns]"]) <= abs(fullGaze.iloc[forward_idx]["timestamp [ns]"] - fullGaze.iloc[index_start]["timestamp [ns]"]):
        print("back_idx is closer/equal than forward idx")
    else:
        print("forward idx is closer than back idx")
    
    print("back idx diff (ms)", abs(fullGaze.iloc[back_idx]["timestamp [ns]"] - fullGaze.iloc[index_start]["timestamp [ns]"])/1e6)
    print("forward idx diff (ms)", abs(fullGaze.iloc[forward_idx]["timestamp [ns]"] - fullGaze.iloc[index_start]["timestamp [ns]"])/1e6) 
    print("forward idx selected as print (default)")
    printI = forward_idx

else:
    printI = index_start
 
print("----------------------------")
if (index_start == printI):
    print("same indices")
else:
    print("different print and start index")

print(index_start)
print(printI)
fullGaze.iloc[printI -5 : printI + 5].head(16) 


----------------------------
same indices
58407
58407


,section id,recording id,timestamp [ns],gaze x [px],gaze y [px],worn,fixation id,blink id,azimuth [deg],elevation [deg]
58402,b8c56a46-b3dc-43c2-a30c-d287553c5ae3,d7a7b5da-afb3-4bc3-91ef-7e0ab1740c55,1694102000998360419,807.721,558.719,1.0,644.0,NaN,-0.875236,3.102295
58403,b8c56a46-b3dc-43c2-a30c-d287553c5ae3,d7a7b5da-afb3-4bc3-91ef-7e0ab1740c55,1694102001001538419,807.830,556.669,1.0,644.0,NaN,-0.868267,3.234017
58404,b8c56a46-b3dc-43c2-a30c-d287553c5ae3,d7a7b5da-afb3-4bc3-91ef-7e0ab1740c55,1694102001007792419,806.705,556.077,1.0,NaN,133.0,-0.940666,3.272069
58405,b8c56a46-b3dc-43c2-a30c-d287553c5ae3,d7a7b5da-afb3-4bc3-91ef-7e0ab1740c55,1694102001014012419,802.210,550.631,1.0,NaN,133.0,-1.230044,3.621861
58406,b8c56a46-b3dc-43c2-a30c-d287553c5ae3,d7a7b5da-afb3-4bc3-91ef-7e0ab1740c55,1694102001017388419,790.324,548.407,1.0,NaN,133.0,-1.994824,3.764135
58407,b8c56a46-b3dc-43c2-a30c-d287553c5ae3,d7a7b5da-afb3-4bc3-91ef-7e0ab1740c55,1694102001023467419,747.934,545.521,1.0,NaN,133.0,-4.721877,3.944930
58408,b8c56a46-b3dc-43c2-a30c-d287553c5ae3,d7a7b5da-afb3-4bc3-91ef-7e0ab1740c55,1694102001026547419,706.889,542.108,1.0,NaN,133.0,-7.361277,4.155552
58409,b8c56a46-b3dc-43c2-a30c-d287553c5ae3,d7a7b5da-afb3-4bc3-91ef-7e0ab1740c55,1694102001032782419,657.162,515.901,1.0,NaN,133.0,-10.570445,5.812807
58410,b8c56a46-b3dc-43c2-a30c-d287553c5ae3,d7a7b5da-afb3-4bc3-91ef-7e0ab1740c55,1694102001039151419,613.188,476.682,1.0,NaN,133.0,-13.438314,8.279391
58411,b8c56a46-b3dc-43c2-a30c-d287553c5ae3,d7a7b5da-afb3-4bc3-91ef-7e0ab1740c55,1694102001042259419,588.171,457.096,1.0,NaN,133.0,-15.080036,9.496497


In [260]:
print(fullGaze.iloc[index_start]["fixation id"])
sess_mask = allLabledData["session"].astype(str).str.startswith(mainSlicing.iloc[index]['file'])
sess_df = allLabledData.loc[sess_mask].copy()
sess_df.head(10)

nan


,session,session_duration,fixation_id,start_timestamp_ns,end_timestamp_ns,duration_ms,fixation_x,fixation_y,fixation_sum_label,fixation_pXc_label,level_sum_annotation,level_pXc_annotation,is_outlier
22514,Expl_5_ET_2_2023-09-07_18-48-26_ET,6.533160e+11,568,1694101977610826419,1694101977851501419,240,864.279,558.681,person,person,1.0,1.0,False
22515,Expl_5_ET_2_2023-09-07_18-48-26_ET,6.533160e+11,569,1694101977992166419,1694101978067139419,74,1270.190,854.295,street,street,1.0,1.0,False
22516,Expl_5_ET_2_2023-09-07_18-48-26_ET,6.533160e+11,570,1694101978185829419,1694101978517253419,331,982.939,954.116,street,street,1.0,1.0,False
22517,Expl_5_ET_2_2023-09-07_18-48-26_ET,6.533160e+11,571,1694101978567181419,1694101978767107419,199,952.718,895.967,street,street,1.0,1.0,False
22518,Expl_5_ET_2_2023-09-07_18-48-26_ET,6.533160e+11,572,1694101978823453419,1694101978973427419,149,1045.220,821.893,street,street,1.0,1.0,False
22519,Expl_5_ET_2_2023-09-07_18-48-26_ET,6.533160e+11,573,1694101979017809419,1694101979142058419,124,976.340,690.786,street,street,1.0,1.0,False
22520,Expl_5_ET_2_2023-09-07_18-48-26_ET,6.533160e+11,574,1694101979223337419,1694101979348402419,125,1207.474,533.242,street,street,1.0,1.0,False
22521,Expl_5_ET_2_2023-09-07_18-48-26_ET,6.533160e+11,575,1694101979448363419,1694101979614075419,165,1174.131,709.044,street,street,1.0,1.0,False
22522,Expl_5_ET_2_2023-09-07_18-48-26_ET,6.533160e+11,576,1694101979657732419,1694101979838981419,181,1026.194,733.347,street,street,1.0,1.0,False
22523,Expl_5_ET_2_2023-09-07_18-48-26_ET,6.533160e+11,577,1694101979889027419,1694101980039010419,149,1096.537,759.050,street,street,1.0,1.0,False


In [261]:
selectedStartTS = printI

mainSlicing.at[index, 'et_start'] = selectedStartTS
mainSlicing.at[index, 'et_start_ns'] = fullGaze.iloc[selectedStartTS]['timestamp [ns]']

mainSlicing.head(13)

,file,session,ET,Paula_start,Paula_start_ns,Paula_end,Paula_end_ns,GPS_sync_start,GPS_sync_start_ns,GPS_sync_end,GPS_sync_end_ns,et_start,et_start_ns,et_end,et_end_ns,experimentStart_worldCFrame,experimentEnd_worldCFrame,experimentStart_worldCFrame_ns,experimentEnd_worldCFrame_ns,Old_et_start,Old_et_start_ns,Old_et_end,Old_et_end_ns
0,Expl_1_ET_1_2023-09-05_11-56-16_ET,1,1,13670,1693904632544755555,32920,1693905274388766666,13670,1693904632544755555,32923,1693905274488733333,96373.0,1693904660577857803,217513.0,1693905267619776803,14509.0,32717.0,1693904660521111111,1693905267621355555,96341.0,1.693905e+18,218604.0,1.693905e+18
1,Expl_1_ET_2_2023-09-05_12-34-24_ET,1,2,9880,1693906794272600000,28805,1693907425280677777,9893,1693906794704611111,28810,1693907425444700000,71073.0,1693906821736889301,190910.0,1693907422101971301,10700.0,28710.0,1693906821612611111,1693907422112644444,65685.0,1.693907e+18,191257.0,1.693907e+18
2,Expl_1_ET_3_2023-09-05_13-10-01_ET,1,3,8060,1693908870728877777,29530,1693909586593522222,8075,1693908871229344444,29530,1693909586593522222,59373.0,1693908900178014929,195045.0,1693909580093629929,8940.0,29340.0,1693908900068955555,1693909580256966666,53609.0,1.693909e+18,196254.0,1.693910e+18
3,Expl_2_ET_1_2023-09-06_10-36-37_ET,2,1,11600,1693986184403811111,30680,1693986820579733333,11613,1693986184835755555,30686,1693986820779733333,81857.0,1693986209353123206,203312.0,1693986818631284206,12348.0,30622.0,1693986209344011111,1693986818643888888,76972.0,1.693986e+18,203626.0,1.693987e+18
4,Expl_2_ET_2_2023-09-06_11-08-36_ET,2,2,8620,1693988004851122222,27960,1693988649695222222,8624,1693988004987155555,27979,1693988650331144444,62423.0,1693988031092340167,185098.0,1693988645782917167,9402.0,27848.0,1693988030927133333,1693988645963177777,57234.0,1.693988e+18,185769.0,1.693989e+18
5,Expl_2_ET_3_2023-09-06_11-39-21_ET,2,3,12160,1693989967720244444,31450,1693990610896299999,12165,1693989967884288888,31457,1693990611128255555,86050.0,1693989994565637975,208819.0,1693990609762533975,12964.0,31450.0,1693989994524300000,1693990610896299999,80751.0,1.693990e+18,208819.0,1.693991e+18
6,Expl_3_ET_1_2023-09-06_13-24-43_ET,3,1,8890,1693996180867344444,28170,1693996823707288888,8900,1693996181199322222,28177,1693996823943277777,66578.0,1693996219236723793,187253.0,1693996823699330793,10036.0,28170.0,1693996219075611111,1693996823707288888,58985.0,1.693996e+18,187253.0,1.693997e+18
7,Expl_3_ET_2_2023-09-06_13-57-57_ET,3,2,8710,1693998168175677777,27720,1693998802015900000,8726,1693998168707433333,27728,1693998802283477777,62520.0,1693998191818152522,183969.0,1693998800180711522,9413.0,27671.0,1693998191615244444,1693998800383222222,57920.0,1.693998e+18,184248.0,1.693999e+18
8,Expl_3_ET_3_2023-09-06_14-28-39_ET,3,3,7640,1693999974960144444,26950,1694000618804288888,7653,1693999975392355555,26955,1694000618972355555,55827.0,1694000000720868204,178518.0,1694000615468059204,8411.0,26850.0,1694000000668477777,1694000615468288888,50805.0,1.694000e+18,178817.0,1.694001e+18
9,Expl_4_ET_1_2023-09-06_18-31-33_ET,4,1,8600,1694014580757844444,27820,1694015221735855555,8607,1694014580993900000,27820,1694015221735855555,60819.0,1694014600729755360,184231.0,1694015219545249360,9192.0,27761.0,1694014600631866666,1694015219767888888,56933.0,1.694015e+18,184654.0,1.694015e+18


In [262]:
mainSlicing.to_csv(os.path.join(slicingpath,"mainSlicingList.csv"), index=False)

## do end slicing

In [280]:
# do end slicing first
# determine the base timestamp
# use GPS synchro timestamp, unless Paulas Timestamp is later --> larger

baseEnd_TS = mainSlicing.at[index,'experimentEnd_worldCFrame_ns']


maskE = fullGaze["timestamp [ns]"] >= baseEnd_TS
index_end = maskE.idxmax()          # returns the position of the **first True**

print(index_end)

if abs(baseEnd_TS - fullGaze.iloc[index_end]['timestamp [ns]']) <= abs(baseEnd_TS - fullGaze.iloc[index_end-1]['timestamp [ns]']):
    print("keep current index and timestamp")

else: 
    print("row earlier (index -1) is actually closer to the world ts")
    index_end = index_end-1
    print(index_end)

174817
row earlier (index -1) is actually closer to the world ts
174816


In [281]:
if pd.notna(fullGaze.iloc[index_end]["fixation id"]):
    print("fixation ID at end index---------------")
    print(fullGaze.iloc[index_end]["fixation id"])
    back_idx, forward_idx, closestI = closest_boundary_index(fullGaze, idx=index_end, col="fixation id")
    print(back_idx, forward_idx, closestI)

    print("differences back idx minus original one", back_idx - index_end)
    print("differences front idx minus original one", forward_idx - index_end)

    print("differences back idx minus original one", back_idx - index_end)
    print("differences front idx minus original one", forward_idx - index_end)

    if abs(fullGaze.iloc[back_idx]["timestamp [ns]"] - fullGaze.iloc[index_end]["timestamp [ns]"]) <= abs(fullGaze.iloc[forward_idx]["timestamp [ns]"] - fullGaze.iloc[index_end]["timestamp [ns]"]):
        print("back_idx is closer/equal than forward idx")
    else:
        print("forward idx is closer than back idx")
    
    print("back idx diff (ms)", abs(fullGaze.iloc[back_idx]["timestamp [ns]"] - fullGaze.iloc[index_end]["timestamp [ns]"])/1e6)
    print("forward idx diff (ms)", abs(fullGaze.iloc[forward_idx]["timestamp [ns]"] - fullGaze.iloc[index_end]["timestamp [ns]"])/1e6) 
    print("back idx selected as print (default)")

    printI = back_idx

else:
    printI = index_end

print("----------------------------")
if (index_end == printI):
    print("same indices")
else:
    print("different print and end index")
    
print(index_end)
print(printI)
fullGaze.iloc[printI -5 : printI + 5].head(16) 

fixation ID at end index---------------
2343.0
174729 174853 174853
differences back idx minus original one -87
differences front idx minus original one 37
differences back idx minus original one -87
differences front idx minus original one 37
forward idx is closer than back idx
back idx diff (ms) 434.308
forward idx diff (ms) 184.729
back idx selected as print (default)
----------------------------
different print and end index
174816
174729


,section id,recording id,timestamp [ns],gaze x [px],gaze y [px],worn,fixation id,blink id,azimuth [deg],elevation [deg]
174724,18a4e6ae-bb0f-4998-9c6a-9d8378364765,82923e39-34c1-42d3-af2e-5b1c2c24660b,1694016722044588994,592.244,822.041,1.0,NaN,437.0,-14.966216,-13.640550
174725,18a4e6ae-bb0f-4998-9c6a-9d8378364765,82923e39-34c1-42d3-af2e-5b1c2c24660b,1694016722047704994,583.989,835.473,1.0,NaN,437.0,-15.541408,-14.477784
174726,18a4e6ae-bb0f-4998-9c6a-9d8378364765,82923e39-34c1-42d3-af2e-5b1c2c24660b,1694016722053936994,578.203,846.879,1.0,NaN,437.0,-15.953913,-15.189111
174727,18a4e6ae-bb0f-4998-9c6a-9d8378364765,82923e39-34c1-42d3-af2e-5b1c2c24660b,1694016722060712994,582.464,859.858,1.0,NaN,437.0,-15.716388,-16.016444
174728,18a4e6ae-bb0f-4998-9c6a-9d8378364765,82923e39-34c1-42d3-af2e-5b1c2c24660b,1694016722063788994,582.946,872.814,1.0,NaN,437.0,-15.728373,-16.835849
174729,18a4e6ae-bb0f-4998-9c6a-9d8378364765,82923e39-34c1-42d3-af2e-5b1c2c24660b,1694016722069587994,582.137,883.880,1.0,NaN,437.0,-15.821032,-17.533361
174730,18a4e6ae-bb0f-4998-9c6a-9d8378364765,82923e39-34c1-42d3-af2e-5b1c2c24660b,1694016722075795994,588.573,896.479,1.0,2343.0,437.0,-15.442562,-18.341816
174731,18a4e6ae-bb0f-4998-9c6a-9d8378364765,82923e39-34c1-42d3-af2e-5b1c2c24660b,1694016722078898994,593.666,905.295,1.0,2343.0,437.0,-15.138477,-18.909060
174732,18a4e6ae-bb0f-4998-9c6a-9d8378364765,82923e39-34c1-42d3-af2e-5b1c2c24660b,1694016722085576994,605.119,915.995,1.0,2343.0,437.0,-14.418256,-19.608304
174733,18a4e6ae-bb0f-4998-9c6a-9d8378364765,82923e39-34c1-42d3-af2e-5b1c2c24660b,1694016722088346994,609.990,914.079,1.0,2343.0,437.0,-14.087121,-19.496345


In [282]:
print(fullGaze.iloc[index_end]["fixation id"])
sess_mask = allLabledData["session"].astype(str).str.startswith(mainSlicing.iloc[index]['file'])
sess_df = allLabledData.loc[sess_mask].copy()
# sess_df.iloc[-130: -100].head(30) 
sess_df.tail()

2343.0


,session,session_duration,fixation_id,start_timestamp_ns,end_timestamp_ns,duration_ms,fixation_x,fixation_y,fixation_sum_label,fixation_pXc_label,level_sum_annotation,level_pXc_annotation,is_outlier
20765,Expl_4_ET_2_2023-09-06_18-57-24_ET,6.396400e+11,2341,1694016721635201994,1694016721753934994,118,884.813,179.652,building,building,1.0,1.0,False
20766,Expl_4_ET_2_2023-09-06_18-57-24_ET,6.396400e+11,2342,1694016721785164994,1694016721894537994,109,813.987,189.480,sky,sky,1.0,1.0,False
20767,Expl_4_ET_2_2023-09-06_18-57-24_ET,6.396400e+11,2343,1694016722075795994,1694016722688624994,612,831.720,611.764,ambiguous,ambiguous,4.0,4.0,True
20768,Expl_4_ET_2_2023-09-06_18-57-24_ET,6.396400e+11,2344,1694016722719564994,1694016722894610994,175,916.171,479.356,person,person,1.0,1.0,False
20769,Expl_4_ET_2_2023-09-06_18-57-24_ET,6.396400e+11,2345,1694016722928973994,1694016723128963994,199,970.102,528.383,building,building,1.0,1.0,False


In [270]:
selectedEndTS = printI

mainSlicing.at[index, 'et_end'] = selectedEndTS
mainSlicing.at[index, 'et_end_ns'] = fullGaze.iloc[selectedEndTS]['timestamp [ns]']

mainSlicing.head(13)

,file,session,ET,Paula_start,Paula_start_ns,Paula_end,Paula_end_ns,GPS_sync_start,GPS_sync_start_ns,GPS_sync_end,GPS_sync_end_ns,et_start,et_start_ns,et_end,et_end_ns,experimentStart_worldCFrame,experimentEnd_worldCFrame,experimentStart_worldCFrame_ns,experimentEnd_worldCFrame_ns,Old_et_start,Old_et_start_ns,Old_et_end,Old_et_end_ns
0,Expl_1_ET_1_2023-09-05_11-56-16_ET,1,1,13670,1693904632544755555,32920,1693905274388766666,13670,1693904632544755555,32923,1693905274488733333,96373.0,1693904660577857803,217513.0,1693905267619776803,14509.0,32717.0,1693904660521111111,1693905267621355555,96341.0,1.693905e+18,218604.0,1.693905e+18
1,Expl_1_ET_2_2023-09-05_12-34-24_ET,1,2,9880,1693906794272600000,28805,1693907425280677777,9893,1693906794704611111,28810,1693907425444700000,71073.0,1693906821736889301,190910.0,1693907422101971301,10700.0,28710.0,1693906821612611111,1693907422112644444,65685.0,1.693907e+18,191257.0,1.693907e+18
2,Expl_1_ET_3_2023-09-05_13-10-01_ET,1,3,8060,1693908870728877777,29530,1693909586593522222,8075,1693908871229344444,29530,1693909586593522222,59373.0,1693908900178014929,195045.0,1693909580093629929,8940.0,29340.0,1693908900068955555,1693909580256966666,53609.0,1.693909e+18,196254.0,1.693910e+18
3,Expl_2_ET_1_2023-09-06_10-36-37_ET,2,1,11600,1693986184403811111,30680,1693986820579733333,11613,1693986184835755555,30686,1693986820779733333,81857.0,1693986209353123206,203312.0,1693986818631284206,12348.0,30622.0,1693986209344011111,1693986818643888888,76972.0,1.693986e+18,203626.0,1.693987e+18
4,Expl_2_ET_2_2023-09-06_11-08-36_ET,2,2,8620,1693988004851122222,27960,1693988649695222222,8624,1693988004987155555,27979,1693988650331144444,62423.0,1693988031092340167,185098.0,1693988645782917167,9402.0,27848.0,1693988030927133333,1693988645963177777,57234.0,1.693988e+18,185769.0,1.693989e+18
5,Expl_2_ET_3_2023-09-06_11-39-21_ET,2,3,12160,1693989967720244444,31450,1693990610896299999,12165,1693989967884288888,31457,1693990611128255555,86050.0,1693989994565637975,208819.0,1693990609762533975,12964.0,31450.0,1693989994524300000,1693990610896299999,80751.0,1.693990e+18,208819.0,1.693991e+18
6,Expl_3_ET_1_2023-09-06_13-24-43_ET,3,1,8890,1693996180867344444,28170,1693996823707288888,8900,1693996181199322222,28177,1693996823943277777,66578.0,1693996219236723793,187253.0,1693996823699330793,10036.0,28170.0,1693996219075611111,1693996823707288888,58985.0,1.693996e+18,187253.0,1.693997e+18
7,Expl_3_ET_2_2023-09-06_13-57-57_ET,3,2,8710,1693998168175677777,27720,1693998802015900000,8726,1693998168707433333,27728,1693998802283477777,62520.0,1693998191818152522,183969.0,1693998800180711522,9413.0,27671.0,1693998191615244444,1693998800383222222,57920.0,1.693998e+18,184248.0,1.693999e+18
8,Expl_3_ET_3_2023-09-06_14-28-39_ET,3,3,7640,1693999974960144444,26950,1694000618804288888,7653,1693999975392355555,26955,1694000618972355555,55827.0,1694000000720868204,178518.0,1694000615468059204,8411.0,26850.0,1694000000668477777,1694000615468288888,50805.0,1.694000e+18,178817.0,1.694001e+18
9,Expl_4_ET_1_2023-09-06_18-31-33_ET,4,1,8600,1694014580757844444,27820,1694015221735855555,8607,1694014580993900000,27820,1694015221735855555,60819.0,1694014600729755360,184231.0,1694015219545249360,9192.0,27761.0,1694014600631866666,1694015219767888888,56933.0,1.694015e+18,184654.0,1.694015e+18


In [271]:
mainSlicing.to_csv(os.path.join(slicingpath,"mainSlicingList.csv"), index=False)

In [272]:
print(((fullGaze.iloc[selectedEndTS]['timestamp [ns]'] - fullGaze.iloc[selectedStartTS]['timestamp [ns]'])/1e9)/60)

10.467242766666667


## now get a rough overview, of how much data was removed by correcting the slicing points 
to actually exclude the gps calibration both in the beginning and end of the experiment time (instead of including the full gps calibration at the start and sometimes removing all or including parts in the end)

In [299]:
sumOld = mainSlicing['Old_et_end_ns'] - mainSlicing['Old_et_start_ns']
sumNew = mainSlicing['et_end_ns'] - mainSlicing['et_start_ns']

sumDiff = sumOld - sumNew
print((np.nanmin(sumDiff)/1e9))
print((np.nanmax(sumDiff)/1e9))
print((np.nanmean(sumDiff)/1e9))
print((np.nansum(sumDiff)/1e9)/60)

# check samples as well
sumOld = mainSlicing['Old_et_end'] - mainSlicing['Old_et_start']
sumNew = mainSlicing['et_end'] - mainSlicing['et_start']

sumDiff = sumOld - sumNew

print('-------------------')
print(np.nanmin(sumDiff))
print(np.nanmax(sumDiff))
print(np.nanmean(sumDiff))
print(np.nansum(sumDiff))


# print((min(sumDiff)/1e9)/60)

5.676564968
38.037551976
26.2462870136
4.374381168933334
-------------------
1123.0
7593.0
5229.1
52291.0


In [301]:
# remove old data columns and save final list

mainSlicing = mainSlicing.drop(columns=['Old_et_start', 'Old_et_start_ns','Old_et_end', 'Old_et_end_ns'])

mainSlicing.head(13)


,file,session,ET,Paula_start,Paula_start_ns,Paula_end,Paula_end_ns,GPS_sync_start,GPS_sync_start_ns,GPS_sync_end,GPS_sync_end_ns,et_start,et_start_ns,et_end,et_end_ns,experimentStart_worldCFrame,experimentEnd_worldCFrame,experimentStart_worldCFrame_ns,experimentEnd_worldCFrame_ns
0,Expl_1_ET_1_2023-09-05_11-56-16_ET,1,1,13670,1693904632544755555,32920,1693905274388766666,13670,1693904632544755555,32923,1693905274488733333,96373.0,1693904660577857803,217513.0,1693905267619776803,14509.0,32717.0,1693904660521111111,1693905267621355555
1,Expl_1_ET_2_2023-09-05_12-34-24_ET,1,2,9880,1693906794272600000,28805,1693907425280677777,9893,1693906794704611111,28810,1693907425444700000,71073.0,1693906821736889301,190910.0,1693907422101971301,10700.0,28710.0,1693906821612611111,1693907422112644444
2,Expl_1_ET_3_2023-09-05_13-10-01_ET,1,3,8060,1693908870728877777,29530,1693909586593522222,8075,1693908871229344444,29530,1693909586593522222,59373.0,1693908900178014929,195045.0,1693909580093629929,8940.0,29340.0,1693908900068955555,1693909580256966666
3,Expl_2_ET_1_2023-09-06_10-36-37_ET,2,1,11600,1693986184403811111,30680,1693986820579733333,11613,1693986184835755555,30686,1693986820779733333,81857.0,1693986209353123206,203312.0,1693986818631284206,12348.0,30622.0,1693986209344011111,1693986818643888888
4,Expl_2_ET_2_2023-09-06_11-08-36_ET,2,2,8620,1693988004851122222,27960,1693988649695222222,8624,1693988004987155555,27979,1693988650331144444,62423.0,1693988031092340167,185098.0,1693988645782917167,9402.0,27848.0,1693988030927133333,1693988645963177777
5,Expl_2_ET_3_2023-09-06_11-39-21_ET,2,3,12160,1693989967720244444,31450,1693990610896299999,12165,1693989967884288888,31457,1693990611128255555,86050.0,1693989994565637975,208819.0,1693990609762533975,12964.0,31450.0,1693989994524300000,1693990610896299999
6,Expl_3_ET_1_2023-09-06_13-24-43_ET,3,1,8890,1693996180867344444,28170,1693996823707288888,8900,1693996181199322222,28177,1693996823943277777,66578.0,1693996219236723793,187253.0,1693996823699330793,10036.0,28170.0,1693996219075611111,1693996823707288888
7,Expl_3_ET_2_2023-09-06_13-57-57_ET,3,2,8710,1693998168175677777,27720,1693998802015900000,8726,1693998168707433333,27728,1693998802283477777,62520.0,1693998191818152522,183969.0,1693998800180711522,9413.0,27671.0,1693998191615244444,1693998800383222222
8,Expl_3_ET_3_2023-09-06_14-28-39_ET,3,3,7640,1693999974960144444,26950,1694000618804288888,7653,1693999975392355555,26955,1694000618972355555,55827.0,1694000000720868204,178518.0,1694000615468059204,8411.0,26850.0,1694000000668477777,1694000615468288888
9,Expl_4_ET_1_2023-09-06_18-31-33_ET,4,1,8600,1694014580757844444,27820,1694015221735855555,8607,1694014580993900000,27820,1694015221735855555,60819.0,1694014600729755360,184231.0,1694015219545249360,9192.0,27761.0,1694014600631866666,1694015219767888888


In [303]:
mainSlicing.to_csv(os.path.join(slicingpath,"mainSlicingList.csv"), index=False)

# other now irrelevant stuff

## create main slicing list

In [ ]:
# not necessary anymore - since already created before the new processing and manual correction of slicing points

In [72]:
# mainSlicing = pd.DataFrame({
#     "file": slicingList["session"],            # copies the column and its index
# })

# mainSlicing['session'] = mainSlicing['file'].str[5].astype(int)
# mainSlicing['ET'] = mainSlicing['file'].str[10].astype(int)

# mainSlicing['Paula_start'] = slicingList['Paula_start']
# mainSlicing['Paula_start_ns'] = slicingList['Paula_start_ns']
# mainSlicing['Paula_end'] = slicingList['Paula_end']
# mainSlicing['Paula_end_ns'] = slicingList['Paula_end_ns']

# mainSlicing['GPS_sync_start'] = slicingList['GPS_sync_start']
# mainSlicing['GPS_sync_start_ns'] = slicingList['GPS_sync_start_ns']
# mainSlicing['GPS_sync_end'] = slicingList['GPS_sync_end']
# mainSlicing['GPS_sync_end_ns'] = slicingList['GPS_sync_end_ns']

# mainSlicing['et_start'] = pd.Series([pd.NA] * len(mainSlicing), dtype="Int64")
# mainSlicing['et_start_ns'] = pd.Series([pd.NA] * len(mainSlicing), dtype="Int64")

# mainSlicing['et_end'] = pd.Series([pd.NA] * len(mainSlicing), dtype="Int64")
# mainSlicing['et_end_ns'] = pd.Series([pd.NA] * len(mainSlicing), dtype="Int64")

# print(mainSlicing.dtypes)

# mainSlicing.head()

file                 object
session               int32
ET                    int32
Paula_start           int64
Paula_start_ns        int64
Paula_end             int64
Paula_end_ns          int64
GPS_sync_start        int64
GPS_sync_start_ns     int64
GPS_sync_end          int64
GPS_sync_end_ns       int64
et_start              Int64
et_start_ns           Int64
et_end                Int64
et_end_ns             Int64
dtype: object


,file,session,ET,Paula_start,Paula_start_ns,Paula_end,Paula_end_ns,GPS_sync_start,GPS_sync_start_ns,GPS_sync_end,GPS_sync_end_ns,et_start,et_start_ns,et_end,et_end_ns
0,Expl_1_ET_1_2023-09-05_11-56-16_ET,1,1,13670,1693904632544755555,32920,1693905274388766666,13670,1693904632544755555,32923,1693905274488733333,<NA>,<NA>,<NA>,<NA>
1,Expl_1_ET_2_2023-09-05_12-34-24_ET,1,2,9880,1693906794272600000,28805,1693907425280677777,9893,1693906794704611111,28810,1693907425444700000,<NA>,<NA>,<NA>,<NA>
2,Expl_1_ET_3_2023-09-05_13-10-01_ET,1,3,8060,1693908870728877777,29530,1693909586593522222,8075,1693908871229344444,29530,1693909586593522222,<NA>,<NA>,<NA>,<NA>
3,Expl_2_ET_1_2023-09-06_10-36-37_ET,2,1,11600,1693986184403811111,30680,1693986820579733333,11613,1693986184835755555,30686,1693986820779733333,<NA>,<NA>,<NA>,<NA>
4,Expl_2_ET_2_2023-09-06_11-08-36_ET,2,2,8620,1693988004851122222,27960,1693988649695222222,8624,1693988004987155555,27979,1693988650331144444,<NA>,<NA>,<NA>,<NA>


# old script fragments, from when I used Paulas and Lanes timestamps, not needed anymore

In [332]:
# do start slicing first
# determine the base timestamp
# use GPS synchro timestamp, unless Paulas Timestamp is later --> larger
print(mainSlicing.iloc[index]['Paula_start_ns'] - mainSlicing.iloc[index]['GPS_sync_start_ns'])

if  mainSlicing.iloc[index]['Paula_start_ns'] <= mainSlicing.iloc[index]['GPS_sync_start_ns']:

    print("Paulas timestamp is equal or earlier/smaller, so use Lanes Timestamp")
    baseStart_TS = mainSlicing.iloc[index]['GPS_sync_start_ns']

else:
    print("Paulas timestamp is later/larger, so use Paulas Timestamp")
    baseStart_TS = mainSlicing.iloc[index]['Paula_start_ns']

mask1 = fullGaze["timestamp [ns]"] >= baseStart_TS
index_start = mask1.idxmax()          # returns the position of the **first True**
mask1 = fullGaze["timestamp [ns]"] == baseStart_TS
index_start2 = mask1.idxmax()          # returns the position of the **first True**

print(index_start)
print(index_start2)

if abs(baseStart_TS - fullGaze.iloc[index_start]['timestamp [ns]']) <= abs(baseStart_TS - fullGaze.iloc[index_start-1]['timestamp [ns]']):
    print("keep current index and timestamp")

else: 
    print("row earlier (index -1) is actually closer to the world ts")

-364344445
Paulas timestamp is equal or earlier/smaller, so use Lanes Timestamp
47298
0
keep current index and timestamp


In [ ]:
# do end slicing first
# determine the base timestamp
# use GPS synchro timestamp, unless Paulas Timestamp is later --> larger
print(mainSlicing.iloc[index]['GPS_sync_end_ns'] - mainSlicing.iloc[index]['Paula_end_ns'])

if  mainSlicing.iloc[index]['GPS_sync_end_ns'] <= mainSlicing.iloc[index]['Paula_end_ns']:

    print("Lanes timestamp is equal or earlier/smaller, so use Lanes Timestamp")
    baseEnd_TS = mainSlicing.iloc[index]['GPS_sync_end_ns']

else:
    print("Paulas timestamp is earlier/smaller, so use Paulas Timestamp")
    baseEnd_TS = mainSlicing.iloc[index]['Paula_end_ns']


maskE = fullGaze["timestamp [ns]"] >= baseEnd_TS
index_end = maskE.idxmax()          # returns the position of the **first True**
maskE2 = fullGaze["timestamp [ns]"] == baseEnd_TS
index_end2 = maskE2.idxmax()          # returns the position of the **first True**

print(index_end)
print(index_end2)

if abs(baseEnd_TS - fullGaze.iloc[index_end]['timestamp [ns]']) <= abs(baseEnd_TS - fullGaze.iloc[index_end-1]['timestamp [ns]']):
    print("keep current index and timestamp")

else: 
    print("row earlier (index -1) is actually closer to the world ts")